# Retrieving Disturbance Data

The [OPERA DIST-HLS data product](https://lpdaac.usgs.gov/documents/1766/OPERA_DIST_HLS_Product_Specification_V1.pdf) can be used to study the evolution of vegetation loss due to natural and anthropogenic causes. 

Deforestation of the Amazon Rainforest in Brazil is an ongoing challenge [[1](https://www.cfr.org/amazon-deforestation/#/en)]. In this notebook, we examine deforestation over a period of one year in the state of Maranhão, Brazil

In [ ]:
from warnings import filterwarnings
filterwarnings("ignore") # suppress PySTAC warnings

# Data imports
import rioxarray as rio

# Plotting imports
from rasterio.warp import transform_bounds
import geoviews as gv
from geoviews import opts
from rasterio.crs import CRS
import hvplot.xarray  # noqa
gv.extension('bokeh')

# GIS imports
from shapely.geometry import Point
from osgeo import gdal
from rasterio.merge import merge

# misc imports
from datetime import datetime

# STAC imports to retrieve cloud data
from pystac_client import Client

# Local imports
from util_functions import search_to_df, urls_to_dataset, filter_search_by_cc

# GDAL setup for accessing cloud data
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','EMPTY_DIR')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','TIF, TIFF')

In [ ]:
maranhao = (-43.65, -3.00)

ref_crs = CRS.from_epsg(4326)
dst_crs = CRS.from_epsg(3857)
map_bounds = transform_bounds(ref_crs, dst_crs, *Point(*maranhao).buffer(10).bounds)

In [ ]:
# visualize AOI as a sanity check
maranhao_gv = gv.Points([maranhao])

basemap = gv.tile_sources.OSM
plot = (maranhao_gv*basemap).opts(
    opts.Points(
        color='red',
        alpha=0.75,
        size=25,
        width=800,
        height=800,
        xlim=(map_bounds[0], map_bounds[2]),
        ylim=(map_bounds[1], map_bounds[3]))
)
plot

NOTE: The OPERA DIST data product is hosted on [LP DAAC](https://lpdaac.usgs.gov/news/lp-daac-releases-opera-land-surface-disturbance-alert-version-1-data-product/), and this is specified when setting up the PySTAC client to search their catalog of data products in the above code cell.

In [ ]:
# We will search data through the product record
start_date = datetime(year=2022, month=1, day=1)
stop_date = datetime(year=2024, month=3, day=31)

# We open a client instance to search for data, and retrieve relevant data records
STAC_URL = 'https://cmr.earthdata.nasa.gov/stac'

# Setup PySTAC client
# LPCLOUD refers to the LP DAAC cloud environment that hosts earth observation data
catalog = Client.open(f'{STAC_URL}/LPCLOUD/') 

collections = ["OPERA_L3_DIST-ALERT-HLS_V1"]

# We would like to search data for August-September 2023
date_range = f'{start_date.strftime("%Y-%m-%d")}/{stop_date.strftime("%Y-%m-%d")}'

search_opts = {
    'bbox' : Point(*maranhao).buffer(.1).bounds, 
    'collections': collections,
    'datetime' : date_range,
}

search = catalog.search(**search_opts)
results = list(search.items_as_dicts())
print(f"Number of tiles found intersecting given AOI: {len(results)}")

In [ ]:
# let's filter our results so that only scenes with less than 10% cloud cover are returned
results = filter_search_by_cc(results, cloud_threshold=20)

print("Number of results containing less than 20% cloud cover: ", len(results))

In [ ]:
# Load results into a pandas dataframe
granules = search_to_df(results, layer_name='VEG-DIST-STATUS')

# filter to study a single geographic area
granules = granules[granules.tile_id == 'T23MPS']

# load data into an xarray dataset
dataset = urls_to_dataset(granules)

**Values in the VEG-DIST-STATUS of the DIST-ALERT product is interpreted as follows:**

* **0:** No disturbance<br>
* **1:** First detection of disturbance with vegetation cover change <50% <br>
* **2:** Provisional detection of disturbance with vegetation cover change <50% <br>
* **3:** Confirmed detection of disturbance with vegetation cover change <50% <br>
* **4:** First detection of disturbance with vegetation cover change >50% <br>
* **5:** Provisional detection of disturbance with vegetation cover change >50% <br>
* **6:** Confirmed detection of disturbance with vegetation cover change >50% <br>
* **7:** Finished detection of disturbance with vegetation cover change <50% <br>
* **8:** Finished detection of disturbance with vegetation cover change >50% <br>

In [ ]:
# Define color map to generate plot (Red, Green, Blue, Alpha)
COLORS = [(255, 255, 255, .1)] * 256  # Initial set all values to white, with 10% opacity

# We would like to visualze only pixels with confirmed and finished disturbances
for i in [3, 6, 7, 8]:
    COLORS[i] = (255, 0, 0, 1)

In [ ]:
img = dataset.hvplot.image(title = 'Deforestation in Maranhão, Brazil. Disturbance Alerts',
                            x='lon', y='lat', 
                            project=True, rasterize=False,
                            framewise=False, 
                            cmap=COLORS, 
                            colorbar=False,
                            widget_location='bottom',
                            tiles = gv.tile_sources.ESRI,
                            xlabel='Longitude (degrees)',ylabel='Latitude (degrees)',
                            fontscale=1.25,
                            frame_width=1000, frame_height=1000,)

img